In [13]:
# Konlpy의 Okt(twitter)로 토크나이징하기
from konlpy.corpus import kolaw
from konlpy.tag import *
import konlpy

openkoreantext = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in openkoreantext.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [14]:
info_naver = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4


In [5]:
info_naver['text'] = info_naver['genre'] + info_naver['intro']
info_naver.tail(1)

NameError: name 'info' is not defined

In [18]:
vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
).fit(info_naver['intro'])

In [19]:
X = vectorize.transform(info_naver['intro'])


In [22]:
X = vectorize.transform(info_naver['intro'])
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

# search 문장 벡터
srch_vector = vectorize.transform([
    "트레이스, 트러블, 인간.. 그들의 생존을 건 싸움",
])

# print(srch_vector.shape)
# srch_vector.toarray()
# features


# linear_kernel은 두 벡터의 dot product이다.
cosine_similar = linear_kernel(srch_vector,X).flatten()
# 위 코드는 아래와 같은 역할을 한다
# cosine_similar = (srch_vector*X.T).toarray().flatten()

# 유사한 rawdata index
sim_rank_idx = cosine_similar.argsort()[::-1]
# print(sim_rank_idx)

for i in sim_rank_idx:
    if cosine_similar[i] > 0 :
        print('title : {} / Intro : {} / score : {}'.format(info_naver.loc[i, 'title'],info_naver['intro'].values[i],cosine_similar[i]))

title : 심연의 하늘 시즌4 / Intro : 재난으로 파괴된 대한민국. 생존을 위한 인간들의 잔혹한 축제가 벌어진다. / score : 0.37692360480737586

생존을 위한 처절한 몸부림. / score : 0.2720590885528513

쫓고 쫓기는 그들의 스릴 넘치는 이야기, '기사도' / score : 0.210950924679459

세계의 운명을 건 싸움이 지금 시작된다 / score : 0.20478793778183313
title : 하나의 하루 / Intro : 인간소녀를 향한 늑대인간의 사랑 / score : 0.20472670909652188
title : 내일은 럭키곰스타 / Intro : 삼류라 불리는 인형이지만, 인간보다 더 인간적인 그들이 펼치는 배꼽빠지게 슬픈이야기 / score : 0.19827424576658592
title : 팀 피닉스 / Intro : 선과 악이 뒤섞인 세상초능력 범죄자와 초능력 경찰의 치열한 싸움이 시작된다 / score : 0.1923357194974853
title : 악의는 없다 / Intro : 시골 마을, 한 남자가 죽었다.그 죽음의 진실을 파헤치려는 이와 덮으려는 자의 싸움. / score : 0.1913066619459935

이제 적도 아군도 없는 싸움이 펼쳐진다. / score : 0.18537479397572185
title : 용이산다 / Intro : 드래곤이든 인간이든 그저 평범한 판타지 일상물.새로운 용(?)물 등장! / score : 0.1575816938574721
title : 신도림 / Intro : 세상이 무너지고 세워진 지하도시 신도림그곳에 들어가지 못하고 생존한 소년들이 지하도시의 비밀을 파헤친다 / score : 0.15583400135848777


살아남은 자들의 숨막히는 생존기 / score : 0.15382229333970626
title : 환생동물학교 / Intro : 동물들이 인간으로 환생하기 전, 동물의 습성을 버리고 인간 세계에서

In [57]:
test = info_naver.copy()
test['similarity'] = cosine_similar
test[test.title == "심연의 하늘 시즌4"]

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like,similarity
646,679547,심연의 하늘 시즌4,윤인완/김선희,7.01,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2016.06.20,story/thrill,1,58706,재난으로 파괴된 대한민국. 생존을 위한 인간들의 잔혹한 축제가 벌어진다.,3,5.39,29099,12032.67,0.376924


In [8]:
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4


### 1st Modeling Process
- feature select : total_like, mean_score, mean_like, total_comments(평균으로 쓸지 고려)
- feature normalization : 0~1사의 값으로 정규화할 것인가 or 평균 0, 표준편차 1로 만들 것인가.
- 재밌게 본 웹툰 선택시 유사도 측정 --> feature 추가

In [11]:
len(info_naver[info_naver['title'] == '슈퍼트리오 시즌2'])

1

In [12]:
from konlpy.corpus import kolaw
from konlpy.tag import *
import konlpy

openkoreantext = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from sklearn.preprocessing import minmax_scale

In [65]:
# min-max scale
def make_scale(df):
    """
    웹툰 수치를 0~1사이로 통일하기위한 함수
    """
    df = df[['mean_like','total_comments','mean_score']]
    scaled_df = pd.DataFrame()
    for column in df.columns:
        scaled_df[column] = minmax_scale(df[column])
    return scaled_df

# 토크나이저
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in openkoreantext.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 해당 웹툰의 소개글 찾기
def find_intro(df, webtoon_name):
    return df[df.title == webtoon_name].intro # 여기서 나온 intro가 get_similarity의 parameter로 간다.

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

# 소개글 받아서 유사도 score 추출 --> column화
def get_similarity(df,intro, vectorize):
    intro_df = df[['unique_id','title','intro']]
    scale_df = make_scale(df)
    df = pd.concat([intro_df,scale_df], axis=1)
    model = vectorize.fit(intro_df.intro)
    X = model.transform(intro_df.intro)
    vector = model.transform(intro)
    # 유사도
    cosine_similar = linear_kernel(vector,X).flatten()
    cosine_similar = cosine_similar.round(2)
    df['target_similarity'] = cosine_similar
    
    return df    

In [68]:
# 유사도 기준 내림차순
# Test : 네이버 웹툰
webtoon = "부활남"
intro = find_intro(info_naver, webtoon)
df = get_similarity(info_naver, intro, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity
227,670139,부활남,나는 죽으면3일 만에 부활한다!,0.156018,0.035263,0.991653,1.00
667,670147,레코닝,내가 살려면 또 다른 내가 죽어야 한다!,0.080610,0.001916,0.998331,0.34
126,163295,공부하기 좋은 날,죽거나 혹은 공부해야하는 우리 학생들.\r\r\n그들에게 벌어지는 조금은 무서운 이야기.,0.017938,0.018461,0.984975,0.26
385,471286,키드갱 시즌2,키드갱 22권 그 이후!\r\r\n정통 개그만화가 부활한다!,0.004977,0.000728,0.968280,0.23
258,358286,단편 할머니/나는내일죽는다,단편 '할머니' / '나는 내일 죽는다'\r\r\n 태발 작가의 강렬한 단편 두 편.,0.000099,0.001638,0.983306,0.20


**장르 키워드도 같이 intro에 추가해서 돌려야...**

In [77]:
# Total score 기준 내림차순
webtoon = "부활남"
intro = find_intro(info_naver, webtoon)
df = get_similarity(info_naver, intro, vectorize=vectorize)
df['total_recommend_score'] = np.sum(df.iloc[:,3:], axis=1)
df.sort_values(by="total_recommend_score", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity,total_recommend_score
227,670139,부활남,나는 죽으면3일 만에 부활한다!,0.156018,0.035263,0.991653,1.00,2.182933
18,20853,마음의소리,"마사루의 센스를, 이나중의 황당함을 뛰어 넘는다! 솔직 담백 최강의 개그 만화 ! ...",0.071059,1.000000,0.981636,0.00,2.052695
3,680911,무한도전 릴레이툰,무한도전X웹툰작가의 뒤 없이 달리는 릴레이 웹툰이\r\r\n방송과 함께 네이버 웹툰...,1.000000,0.024946,0.919866,0.00,1.944813
197,183559,신의 탑,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,0.186566,0.671172,0.991653,0.03,1.879391
194,570503,연애혁명,"로맨스, 그런 건 우리에게 있을 수가 없어!신개념 개그 로맨스",0.350541,0.454952,0.983306,0.00,1.788798


**합산하면 결과가 별로...다...회차 평균 댓글수로 수정을 해봐야 할 것 같다.**

In [81]:
# 다음 웹툰 intro로 네이버 웹툰 유사도 측정
intro = ["거만한 천재 소년과 심약한 은둔 고수와의 만남, 재능과 노력만으론 최고가 될 수 없다!"]
df = get_similarity(info_naver, intro, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity
624,670145,킬더킹,"왕의 재능이란 무엇일까?왕좌의 새로운 주인, 진정한 왕에 걸맞는 재능을 가려내는 게...",0.019374,0.010132,0.948247,0.28
678,675822,대작,천재 신인작가와 그를 쫓는 이들의 숨막히는 이야기,0.070070,0.000911,0.993322,0.23
203,662774,고수,최강의 콤비가 무협의 전설을 다시 쓰다! 천하제일의 고수 강룡. 그리고 수많은 다...,0.212850,0.073468,0.996661,0.23
451,308801,봄이여 오라,"평범한 일상 속, 평범한 만남.\r\r\n그 안에서 엮이는 조금은 특별한 인연.",0.003284,0.006291,0.989983,0.19
532,697681,목욕의 신 (재),신의 손을 가진 남자최고의 목욕관리사가 되기위한 때밀이 배틀이 시작된다.,0.228599,0.000963,0.996661,0.17
